In [1]:
import multiprocessing
from pathlib import Path
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import cv2
import numpy as np
import imutils
import random
import pickle
import os
import shutil
import time
import Augmentor

In [2]:
def dataset_augmentation(path, nb_imgs):
    # Path to the image dataset
    p = Augmentor.Pipeline(str(path), output_directory=".")

    # Operations to be performed on the images:
    p.rotate90(probability=0.2)
    p.rotate270(probability=0.2)
    p.flip_left_right(probability=0.30)
    p.flip_top_bottom(probability=0.30)
    p.skew_tilt(probability=0.25, magnitude=0.1)
    p.random_distortion(probability=1, grid_width=2, grid_height=2, magnitude=4)

    # Specifyin the number of images to generate
    p.sample(nb_imgs)
    
    print({path}, ": augmentation done for", {nb_imgs}, "more images.")

In [3]:
ROOT_DIR = Path.cwd()

SHAPES = ['Cercles', 'Diamants', 'Hexagones', 'Triangles']

THRESHOLDS = {
    'Cercle2': 0.6,
    'Cercle3': 0.6,
    'Cercle4': 0.6,
    'Cercle5': 0.4,
    'Diamant2': 0.6,
    'Diamant3': 0.6,
    'Diamant4': 0.6,
    'Diamant5': 0.4,
    'Hexagone2': 0.6,
    'Hexagone3': 0.6,
    'Hexagone4': 0.6,
    'Hexagone5': 0.5,
    'Triangle2': 0.6,
    'Triangle3': 0.6,
    'Triangle4': 0.6,
    'Triangle5': 0.6,
}

GROUPS = [2,3,4,5]

path_ensemble_A = Path(ROOT_DIR, 'EnsembleA_H2020')

In [4]:
def get_shapes_markers(shape_dir):
    """
    Prend le nom du pattern en string en parametre
    :return dict avec les paths des images et le marker associer
    exemple:
        {
            "Cercle2":  {
                "marker_path": PosixPath('path/to/markers/C2.jpg'),
                "images": [PosixPath('path/to/images/Cercle2/001_Cercle2.jpg'), ...],
            },
            "Cercle3": {...},
            ...
        }
    """
    paths = [path_ensemble_A/Path(shape_dir, f'{shape_dir[:-1]}{i}') for i in GROUPS]

    sub_shapes = dict()
    for p in paths:
        img = []
        for root, dirs, files in os.walk(str(p)):
            [img.append(f'{p}/{f}') for f in files]
        sub_shapes.update({
            p.name: {
                'marker_path': Path(ROOT_DIR, 'Resources', 'markers', f'{p.name[:1]+p.name[-1:]}.jpg'),
                'images': img,
            }
        })

    return sub_shapes

In [5]:
# RUN THIS CELL TO CREATE ENSEMBLE_B DATABASE
#  TO REVISIT:
#  TWEEKING OF THRESHOLD FOR EACH SHAPE 

ensemble_A = {}

for shape in SHAPES:
    imgs = get_shapes_markers(shape)
    ensemble_A.update({shape: imgs})
    
path_ensemble_B = Path(ROOT_DIR, 'EnsembleB_H2020')

if os.path.exists(str(path_ensemble_B)):
    shutil.rmtree(str(path_ensemble_B), ignore_errors=True)
    time.sleep(1)
path_ensemble_B.mkdir(parents=True, exist_ok=True)


In [6]:
for shape, groups in ensemble_A.items():
    for group, paths in groups.items():
        group_path = path_ensemble_B/Path(shape, group)
        group_path.mkdir(parents=True, exist_ok=True)
        marker = cv2.imread(str(paths['marker_path']), cv2.IMREAD_GRAYSCALE)
        nb_match = 0
        for image_path in paths['images']:
            image = cv2.imread(str(image_path))
            new_path=str(image_path).replace("EnsembleA","EnsembleB")

            bitwise = cv2.bitwise_not(image)
            resized = imutils.resize(bitwise, width=300)
            # ratio = image.shape[0] / float(resized.shape[0])

            #  https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_thresholding/py_thresholding.html
            # convert the resized image to grayscale, blur it slightly,
            # and threshold it
            gray = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
            # blurred = cv2.GaussianBlur(gray, (5, 5), 0)
            # thresh = cv2.threshold(blurred, 160, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C)[1]

# WILL NEED TO EXTRACT IMAGE BEFORE FINDING FEATURES
            img = gray
            w,h = img.shape[::-1]
            base_width = int(w/2)
            i=0
            has_matched = False
            templateRotation=0
#             im_crop=0
            while templateRotation < 350:

                rotated = imutils.rotate(marker, angle=templateRotation)
                templateRotation=templateRotation+10

                while i < base_width-40:
                    if has_matched:
                        break
                    bitwise = cv2.bitwise_not(marker)
                    resized = imutils.resize(bitwise, width=base_width-i)

                    w,h = resized.shape[::-1]
                    i=i+20
                    result = cv2.matchTemplate(img, resized, cv2.TM_CCOEFF_NORMED)
                    loc = np.where(result >= THRESHOLDS[group])

                    for pt in zip(*loc[::-1]):
                        has_matched = True
                        im_crop = img[pt[1]:pt[1]+h, pt[0]:pt[0]+w]
                        cv2.imwrite(new_path, im_crop)
                        nb_match = nb_match + 1
                        break
                        
        print(f'match {group} : {nb_match}/{len(paths["images"])}')

match Cercle2 : 84/311
match Cercle3 : 126/555
match Cercle4 : 92/348
match Cercle5 : 58/212
match Diamant2 : 134/444
match Diamant3 : 95/332
match Diamant4 : 69/252
match Diamant5 : 153/203
match Hexagone2 : 101/339
match Hexagone3 : 79/276
match Hexagone4 : 53/239
match Hexagone5 : 60/271
match Triangle2 : 165/442
match Triangle3 : 78/214
match Triangle4 : 59/237
match Triangle5 : 50/149


In [9]:
dataset_augmentation(path_ensemble_B / "Cercles" / "Cercle2", 50)
dataset_augmentation(path_ensemble_B / "Cercles" / "Cercle3", 50)
dataset_augmentation(path_ensemble_B / "Triangles" / "Triangle3", 50)
dataset_augmentation(path_ensemble_B / "Diamants" / "Diamant3", 50)
dataset_augmentation(path_ensemble_B / "Hexagones" / "Hexagone2", 50)
dataset_augmentation(path_ensemble_B / "Hexagones" / "Hexagone3", 50)

Processing <PIL.Image.Image image mode=L size=110x110 at 0x22100130F08>: 100%|██| 50/50 [00:00<00:00, 294.90 Samples/s]

Initialised with 134 image(s) found.
Output directory set to D:\Ecole\LOG635\Lab2\EnsembleB_H2020\Cercles\Cercle2\..{WindowsPath('D:/Ecole/LOG635/Lab2/EnsembleB_H2020/Cercles/Cercle2')} : augmentation done for {50} more images.



Processing <PIL.Image.Image image mode=L size=70x70 at 0x2217BAECFC8>: 100%|████| 50/50 [00:00<00:00, 557.05 Samples/s]

Initialised with 126 image(s) found.
Output directory set to D:\Ecole\LOG635\Lab2\EnsembleB_H2020\Cercles\Cercle3\..{WindowsPath('D:/Ecole/LOG635/Lab2/EnsembleB_H2020/Cercles/Cercle3')} : augmentation done for {50} more images.



Processing <PIL.Image.Image image mode=L size=70x70 at 0x2210215A508>: 100%|████| 50/50 [00:00<00:00, 626.67 Samples/s]

Initialised with 78 image(s) found.
Output directory set to D:\Ecole\LOG635\Lab2\EnsembleB_H2020\Triangles\Triangle3\..{WindowsPath('D:/Ecole/LOG635/Lab2/EnsembleB_H2020/Triangles/Triangle3')} : augmentation done for {50} more images.



Processing <PIL.Image.Image image mode=L size=90x90 at 0x221021A3688>: 100%|████| 50/50 [00:00<00:00, 611.40 Samples/s]

Initialised with 95 image(s) found.
Output directory set to D:\Ecole\LOG635\Lab2\EnsembleB_H2020\Diamants\Diamant3\..{WindowsPath('D:/Ecole/LOG635/Lab2/EnsembleB_H2020/Diamants/Diamant3')} : augmentation done for {50} more images.



Processing <PIL.Image.Image image mode=L size=70x70 at 0x2210012CAC8>: 100%|████| 50/50 [00:00<00:00, 589.81 Samples/s]

Initialised with 79 image(s) found.
Output directory set to D:\Ecole\LOG635\Lab2\EnsembleB_H2020\Hexagones\Hexagone3\..{WindowsPath('D:/Ecole/LOG635/Lab2/EnsembleB_H2020/Hexagones/Hexagone3')} : augmentation done for {50} more images.
